# Handwriting Recognizer Notebook

This notebook contains the logic for recognizing handwritten English alphabet text from an image file using a Keras model.

It defines a function `recognize_text_from_image(filepath, model_path)` which can be called from another notebook after running this one using `%run`.

**Important:** This notebook requires the Keras model file to be at the specified path.


## 1. Setup and Imports

Imports necessary libraries for handwriting recognition.


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
import traceback

print("Handwriting Recognizer: Libraries imported.")


## 2. Load Model & Define Recognition Function

Loads the Keras model (path currently set to `C:\Users\murta\Desktop\New folder\Handwritten-Alphabets-Recognition\Alphabet_Recognition.keras`) and defines the core recognition function.


In [ ]:
# Define the alphabet mapping
alpha = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")

# Global variable to hold the loaded model to avoid reloading
_loaded_hw_model = None
_loaded_hw_model_path = None

def load_handwriting_model(model_path):
    """Loads the Keras handwriting recognition model."""
    global _loaded_hw_model, _loaded_hw_model_path
    normalized_path = os.path.normpath(model_path)
    
    if _loaded_hw_model is not None and _loaded_hw_model_path == normalized_path:
        return _loaded_hw_model

    try:
        if not os.path.exists(normalized_path):
            print(f"Error: Model file not found at {normalized_path}")
            _loaded_hw_model = None
            _loaded_hw_model_path = None
            return None
            
        _loaded_hw_model = tf.keras.models.load_model(normalized_path)
        _loaded_hw_model_path = normalized_path
        print(f"Handwriting model loaded successfully from {normalized_path}")
        return _loaded_hw_model
    except Exception as e:
        print(f"Error loading handwriting model from {normalized_path}: {e}")
        _loaded_hw_model = None
        _loaded_hw_model_path = None
        return None

def recognize_text_from_image(filepath, model_path):
    """Processes an image file, recognizes handwritten text, and returns the text string."""
    model = load_handwriting_model(model_path)
    if model is None:
        return "Error: Handwriting recognition model could not be loaded."

    normalized_filepath = os.path.normpath(filepath)
    try:
        image = cv2.imread(normalized_filepath)
        if image is None: return f"Error: Could not read image file at {normalized_filepath}"
        blur_image = cv2.medianBlur(image, 7)
        grey = cv2.cvtColor(blur_image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.adaptiveThreshold(grey, 200, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 41, 25)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours: return "No text contours found."
        preprocessed_digits = []
        coords = []
        boundingBoxes = [cv2.boundingRect(c) for c in contours]
        min_contour_area = 10
        filtered_contours = []
        filtered_boundingBoxes = []
        for i, c in enumerate(contours):
            if cv2.contourArea(c) > min_contour_area: filtered_contours.append(c); filtered_boundingBoxes.append(boundingBoxes[i])
        if not filtered_contours: return "No significant text contours found."
        contours_sorted, boundingBoxes_sorted = zip(*sorted(zip(filtered_contours, filtered_boundingBoxes), key=lambda b: b[1][0]))
        for i, c in enumerate(contours_sorted):
            x, y, w, h = boundingBoxes_sorted[i]
            coords.append(x)
            digit = thresh[y:y + h, x:x + w]
            resized_digit = cv2.resize(digit, (18, 18))
            padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)
            preprocessed_digits.append(padded_digit)
        alphabets = []
        for i, digit in enumerate(preprocessed_digits):
            prediction_input = digit.reshape(1, 28, 28, 1) / 255.0
            prediction = model.predict(prediction_input, verbose=0)
            pred_index = np.argmax(prediction)
            pred_char = alpha[pred_index] if 0 <= pred_index < len(alpha) else "."
            alphabets.append((coords[i], pred_char, boundingBoxes_sorted[i][2]))
        final_text = ""
        if alphabets:
            gaps = [alphabets[i + 1][0] - (alphabets[i][0] + alphabets[i][2]) for i in range(len(alphabets) - 1)]
            median_gap = np.median(gaps) if gaps else 10
            space_threshold = max(median_gap * 1.2, 8)
            last_x_end = 0
            for i, (x_coord, char, width) in enumerate(alphabets):
                if i > 0 and (x_coord - last_x_end) > space_threshold: final_text += " "
                final_text += char
                last_x_end = x_coord + width
        return final_text.strip()
    except Exception as e: print(traceback.format_exc()); return f"Error during handwriting recognition: {e}"

# Define the model path variable used by the function
handwriting_model_path_for_notebook = r"C:\Users\murta\Desktop\New folder\Handwritten-Alphabets-Recognition\Alphabet_Recognition.keras"

# Pre-load the model when the notebook cell is run (optional, but efficient)
print("Attempting to pre-load handwriting model...")
_ = load_handwriting_model(handwriting_model_path_for_notebook)

print("Handwriting Recognizer: Functions defined and model pre-load attempted.")
